# Lab 2 - Layouts in Shiny 


Dashboards summarize important information in a visual display that can fit in single computer screen so that in can be monitored at a glance. 
Depending on the audience and the goal, we can broadly classify dashboards as data exploration tools or data presentation tools. 
If the intended audience is data analysts (ourselves or our customer), 
data exploration becomes the key aspect of dashboard design. 
The users should be able to manipulate data, try new hypotheses, 
and visualize results. 
If the audience is the end users with less analytical ability but are in the business decision making, 
dashboard designs should emphasize data presentation aspects. 
There should be a clear message and enough supporting evidence (visual and otherwise) to facilitate decision making.
This type of dashboard should guide the decision makers to make smarter choices based on the analyzed data. 
For the purposes of data science, we utilize both the data exploration and data presentation aspects of interactive dashboards. 

Layouts play an important role in making dashboard designs more effective so that data explorers or decision makers can interact with the data effortlessly. 
We have seen in the data visualization class how to make visualizations more effective, 
in this course we will work on designing effective dashboards and interactive visualizations. 

In this lab notebook, we will continue studying the basics of Shiny, 
specifically how to control the layout of our Shiny apps. 
To make the design more useful and intuitive, we can add a variety of components to our Shiny apps such as tabs and navigation bars in addition to the panels we have seen in the previous examples.  

### Single-file Shiny Apps

Before we start looking at layout options, we'll see how we can have a Shiny app in a **single file**. 
So far, we have created `ui.R` and `server.R` to deploy a Shiny app; 
newer versions of Shiny supports single-file applications where you no longer need to create separate files for your app. 
You can simply create a file called **`app.R`** that contains both the server and UI components. 
**Your `app.R` file should call shinyApp() with an appropriate UI object and server function, as demonstrated below**:

---
```
# This is the UI component.

ui <- fluidPage(
  titlePanel("Hello Shiny!"),
  sidebarLayout(
    sidebarPanel(
      sliderInput("obs", "Number of observations:", min = 10, max = 500, value = 100)
    ),
    mainPanel(plotOutput("distPlot"))
  )
)

# This is the server component. 

server <- function(input, output) {
  output$distPlot <- renderPlot({
    hist(rnorm(input$obs), col = 'darkgray', border = 'white')
  })
}

# This lets Shiny know where UI and server functions are 
# (you could use different names like myUI, xyzserver, etc.) 

shinyApp(ui = ui, server = server)
```
---

This is much simpler than creating two separate files. **Copy and paste this to the app.R file in ```module2-single``` folder and deploy it using the following code: **



In [ ]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module2-single" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

### Sidebar layout 

A basic Shiny app can consist of a sidebar panel and a main panel as we have seen in the previous examples. The above example shows the two panels; ```sidebarPanel``` and ```mainpanel``` inside the ```sidebarLayout```.

<img src="../images/sidebarLayout.png">

Sidebar panel provides widgets for input so that user can interact with the way data is visualized. Main panel displays the visualizations and related information. 


### Grid layout 

The sidebar layout in the above example is actually created by using lower level grid functions. We can have a detailed control of how thigs are placed in the layout. Rows are created by the ```fluidRow()``` function, and they include columns by using the ```column()``` function. 
For example, we can reimplement the sidebar layout by using these functions: 

---
```
ui <- fluidPage(

  titlePanel("Hello Shiny!"),

  fluidRow(
  
    column(4,
      wellPanel(
        sliderInput("obs", "Number of observations:",  
                    min = 1, max = 1000, value = 500)
      )       
    ),

    column(8,
      plotOutput("distPlot")
    )
  )
)

server <- function(input, output) {
  output$distPlot <- renderPlot({
    hist(rnorm(input$obs), col = 'darkgray', border = 'white')
  })
}

shinyApp(ui = ui, server = server)
```
---

First parameter to the ```column()``` function specifies the width of the column; **given numbers should add up to 12**. ```wellPanel()``` creates a panel with a slightly inset border and grey background. The above code recreates the ```sidebarLayout``` we have seen previously. 

Here is an example of grid layout with a plot at the top and three columns at the bottom: 

---
```
library(shiny)
library(ggplot2)

dataset <- diamonds

# UI 

ui <- fluidPage(

  title = "Diamonds Explorer",  
  plotOutput('plot'),
  hr(),
  fluidRow(
    column(3,
      h4("Diamonds Explorer"),
      sliderInput('sampleSize', 'Sample Size', 
                  min=1, max=nrow(dataset), value=min(1000, nrow(dataset)), 
                  step=500, round=0),
      br(),
      checkboxInput('jitter', 'Jitter'),
      checkboxInput('smooth', 'Smooth')
    ),
    column(4, offset = 1,
      selectInput('x', 'X', names(dataset)),
      selectInput('y', 'Y', names(dataset), names(dataset)[[2]]),
      selectInput('color', 'Color', c('None', names(dataset)))
    ),
    column(4,
      selectInput('facet_row', 'Facet Row', c(None='.', names(dataset))),
      selectInput('facet_col', 'Facet Column', c(None='.', names(dataset)))
    )
  )
)

# server

server <- function(input, output) {
 
  dataset <- reactive({
    diamonds[sample(nrow(diamonds), input$sampleSize),]
  })
 
  output$plot <- renderPlot({
    
    p <- ggplot(dataset(), aes_string(x=input$x, y=input$y)) + geom_point()
    
    if (input$color != 'None')
      p <- p + aes_string(color=input$color)
    
    facets <- paste(input$facet_row, '~', input$facet_col)
    if (facets != '. ~ .')
      p <- p + facet_grid(facets)
    
    if (input$jitter)
      p <- p + geom_jitter()
    if (input$smooth)
      p <- p + geom_smooth()
    
    print(p)
    
  })
  
}

shinyApp(ui = ui, server = server)

```
---

This is how it looks like as a running app:
<img src="../images/columnLayout.png">



### Tabsets

When applications need to subdivide their user interface into discrete sections, we can do this using the ```tabsetPanel``` as shown below:

---
```
ui <- fluidPage(

  titlePanel("Tabsets"),

  sidebarLayout(
    
    sidebarPanel(
      # Inputs excluded for brevity
    ),
  
    mainPanel(
      tabsetPanel(
        tabPanel("Plot", plotOutput("plot")), 
        tabPanel("Summary", verbatimTextOutput("summary")), 
        tabPanel("Table", tableOutput("table"))
      )
    )
  )
)

# server code here ...
```
---
<img src="../images/tabsets.png">

Each tabset can visualize different aspects of data that is loosely connected to each other so that they do not need to be displayed together. We will see an example of this in the practices. 

### Navlists

If you have too many tabPanels, the ```navlistPanel``` is a good alternative. A navlist presents the components as a sidebar list rather than using tabs in a horizontal setting. It can also have section headings and separators to make use of perceptual grouping. Here is an example: 

---
```
ui <- fluidPage(
  
  titlePanel("Application Title"),
  
  navlistPanel(
    "Header A",
    tabPanel("Component 1"),
    tabPanel("Component 2"),
    "Header B",
    tabPanel("Component 3"),
    tabPanel("Component 4"),
    "-----",
    tabPanel("Component 5")
  )
)
```
---
Note that the tabPanels are empty to keep the example uncluttered, typically they would include additional UI elements, such as the outputs in panels. This is how it looks like:

<img src="../images/navlistPanel.png">

### Navbar pages

If you want to create a Shiny application that consists of multiple distinct subcomponents each with their own sidebar, tabsets, or other layout constructs, you can use the ```navbarPage()``` function to create a Navbar at the top. For example:

---
```
ui <- navbarPage("My Application",
  tabPanel("Component 1"),
  tabPanel("Component 2"),
  tabPanel("Component 3")
)

```
---
Again, here tabPanels can contain their own UI elements. This is how it looks like:

<img src="../images/navbarPage.png">

### Fluid Grid system

To create the user interface as a web page, 
Shiny utilizes the well-known Bootstrap library in the background. 
There are two types of grids in Bootstrap; fluid and fixed. 
So far, we have seen the fluid grid system in the examples. 
Both grid systems use a flexibly sub-dividable 12-column grid for layout. The fluid system always occupies the full width of the web page and resizes its components dynamically as the size of the page changes. 

This is a useful behavior to adapt the layout to different types of screens like desktop, tablet, or phone with changing orientations. 
The layout is "fluid", it automatically adapts to the screen size and orientation. We should use this system by default. 

The fixed system occupies a fixed width of 940 pixels by default and may assume other widths when Bootstrap responsive layout kicks in (e.g. when on a tablet).

 To create a layout based on the fluid system, we use the ```fluidPage()``` function. 
 To create rows within the grid, we use the ```fluidRow()``` function; 
to create columns within rows, we use the ```column()``` function. 
For example, the following piece of code would create a **sidebar column and a main column with a 2/10 ratio in width**:
 
---
```
ui <- fluidPage(
  fluidRow(
    column(2,
      "sidebar"
    ),
    column(10,
      "main"
    )
  )
)
```
---

<img src="../images/fluidGrid.png">
 
We can also **nest** columns inside other columns. 
Each nested level of columns should **add up to 12 columns**. 
This is because the fluid grid uses percentages, not pixels, for setting widths.
 
--- 
```
ui <- fluidPage(
  fluidRow(
    column(12,
      "Fluid 12",
      fluidRow(
        column(6,
          "Fluid 6",
          fluidRow(
            column(6, 
              "Fluid 6"),
            column(6,
              "Fluid 6")
          )
        ),
        column(width = 6,
          "Fluid 6")
      )
    )
  )
)

```
---

<img src="../images/fluidNesting.png">

### Fixed Grid system

The fixed grid system also utilizes 12 columns, and maintains a fixed width of 940 pixels by default. 
If Bootstrap responsive features are enabled (they are by default in Shiny) then the grid will also adapt to be 724px or 1170px wide depending on your viewport (e.g. when on a tablet).

The main benefit of a fixed grid is that it provides stronger guarantees about how users will see the various elements of your UI laid out (this is because it’s not being dynamically laid out according to the width of the browser). 
The main drawback is that it’s a bit more complex to work with. 

In general it is recommended to use fluid grids unless you absolutely require the lower level layout control afforded by a fixed grid. 
Using fixed grids in Shiny works *almost* identically to fluid grids. 
You use the fixedPage() and fixedRow() functions to build the grid. 
Rows can nest, but should always include a set of columns that add up to the number of columns of their parent (*rather than resetting to 12 at each nesting level as they do in fluid grids*).

Here’s the code for a fixed grid version of the simple sidebar layout shown earlier:

---
```
ui <- fixedPage(
  fixedRow(
    column(2,
      "sidebar"
    ),
    column(10,
      "main"
    )
  )
)
```
---
